In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tabular.dl_logs import create_model_identifier, CALMIP_START_INDEX

%reload_ext autoreload
%autoreload 2

SCRIPT_PATH = Path('./scripts/calmip')
LOG_FILE = Path('./logs/calmip_logs.csv')

# args creation from logs

In [2]:
common_args = ["--task_name=long_term_forecast",
               "--is_training= 1",
               "--root_path = /tmpdir/arnaud/dataset/USC/",
               "--checkpoints = /tmpdir/arnaud/checkpoints/",
               "--results_path = /tmpdir/arnaud/results/",
               "--data_path = na",
               "--data = USC",
               "--des = Exp",
               "--batch_size = 32",
               "--train_epochs = 48",
               "--patience= 16",
               "--embed=fixed",
               "--consider_only_offense",
               "--inverse",
               "--itr= 3"]


In [3]:
ldf = pd.read_csv(LOG_FILE, index_col=0)
ldf = ldf[ldf.index >= CALMIP_START_INDEX[2]]
ldf

,task_name,is_training,model_id,model,data,root_path,data_path,features,target,freq,...,nb_params,nb_tr_params,nb_nontr_params,mae,mse,rmse,mape,mspe,fde,results_path
471,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.614937,0.878844,0.937467,inf,inf,1.597670,/tmpdir/arnaud/results
472,long_term_forecast,1,USC_24_24,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.913752,2.036071,1.426909,inf,inf,2.472808,/tmpdir/arnaud/results
473,long_term_forecast,1,USC_48_48,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,1.647258,7.078037,2.660458,inf,inf,4.432114,/tmpdir/arnaud/results
474,long_term_forecast,1,USC_32_32,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,1.195554,3.562750,1.887525,inf,inf,3.252716,/tmpdir/arnaud/results
475,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,...,791809,791809,0,0.607831,0.871429,0.933504,inf,inf,1.594972,/tmpdir/arnaud/results
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,long_term_forecast,1,'USC_24_24',Nonstationary_Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,4228173,1629261,2598912,0.895622,1.989921,1.410645,inf,inf,2.424765,/tmpdir/arnaud/results/
1206,long_term_forecast,1,'USC_24_24',Nonstationary_Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,1787597,488141,1299456,0.905210,1.992312,1.411493,inf,inf,2.414110,/tmpdir/arnaud/results/
1207,long_term_forecast,1,'USC_24_24',Nonstationary_Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,837645,187917,649728,0.937553,2.107247,1.451636,inf,inf,2.439283,/tmpdir/arnaud/results/
1208,long_term_forecast,1,'USC_24_24',Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,MS,OT,h,...,836989,187261,649728,0.767012,1.569087,1.252632,inf,inf,2.256347,/tmpdir/arnaud/results/


In [4]:
ldf['exp_id'] = ldf.apply(lambda row : create_model_identifier(row), axis =1)
edf = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ][['exp_id',"fde","features",'fit_time','epoch']].groupby('exp_id').agg({'fde':'count','features':'count','fit_time':'sum', 'epoch' : 'mean'})
edf['fit_time'] = edf['fit_time'] / 3600
edf

,fde,features,fit_time,epoch
exp_id,,,,
DLinear_S_16-16_el2_dm512_nh8_dl1_ff2048,3,3,0.734414,38.000000
LSTM_S_16-16_el1_dm256_nh8_dl1_ff512,3,3,0.808700,46.000000
LSTM_S_16-16_el1_dm64_nh8_dl1_ff64,3,3,0.796625,45.333333
LSTM_S_16-16_el2_dm256_nh8_dl1_ff512,3,3,0.804517,36.666667
LSTM_S_16-16_el2_dm64_nh8_dl1_ff64,3,3,0.848978,46.000000
LSTM_S_16-16_el2_dm64_nh8_dl2_ff64,3,3,0.897356,46.666667
Nonstationary_Transformer_S_16-16_el2_dm128_nh8_dl1_ff128,3,3,4.479383,34.333333
Nonstationary_Transformer_S_16-16_el2_dm256_nh8_dl1_ff256,3,3,4.488947,37.333333
Nonstationary_Transformer_S_16-16_el2_dm64_nh4_dl1_ff64,3,3,4.476750,39.666667


In [5]:
time_limits = {
    'LSTM' : '02:00:00',
    'Transformer' :'08:00:00',
    'iTransformer' :'02:00:00',
    'PatchTST' :'06:00:00',
    'Nonstationary_Transformer' :'06:00:00',
    'TimesNet' :'06:00:00',
    'DLinear' :'02:00:00'    
}

In [6]:
tmp_df = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ].copy()

configs = []
config_df = tmp_df[['model','dropout','learning_rate','optimizer', "wd","e_layers",'d_model','n_heads', 'd_layers', 'd_ff']].copy()
configs = [v for v in config_df.drop_duplicates().T.to_dict().values()]
len(configs)    

27

In [7]:
configs_list = [[f"--{k}={v}" for k,v in config.items()]for config in configs]
configs_list

[['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=256',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=512'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=1',
  '--d_model=256',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=512'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=1',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=64'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=64'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=2',
  '--d_ff=64'],
 ['--model=Transformer',
  '--dropout=0.1',
  '--le

In [8]:
features_set = {
   'S' : ["--features=S", "--enc_in=1", "--dec_in=1", "--c_out=1"],
   'MS' : ["--features=MS", "--enc_in=61", "--dec_in=61", "--c_out=61"]
}
time_horizons = [16,24,32,48,64,96]


In [9]:
exp_count = ldf.groupby('exp_id').count()['features']
exp_count

exp_id
DLinear_MS_16-16_el2_dm512_nh8_dl1_ff2048       3
DLinear_MS_24-24_el2_dm512_nh8_dl1_ff2048       3
DLinear_MS_32-32_el2_dm512_nh8_dl1_ff2048       3
DLinear_MS_48-48_el2_dm512_nh8_dl1_ff2048       3
DLinear_MS_64-64_el2_dm512_nh8_dl1_ff2048       3
                                               ..
iTransformer_S_96-96_el2_dm512_nh4_dl1_ff512    3
iTransformer_S_96-96_el3_dm256_nh4_dl1_ff256    3
iTransformer_S_96-96_el3_dm512_nh4_dl1_ff512    3
iTransformer_S_96-96_el4_dm256_nh8_dl1_ff256    3
iTransformer_S_96-96_el4_dm512_nh8_dl1_ff512    3
Name: features, Length: 247, dtype: int64

In [10]:
with open(SCRIPT_PATH / 'jobs_configs.txt', "w") as f:
    i = 0
    for th in time_horizons :
        th_args = [f"--model_id='USC_{th}_{th}'", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//4}"]
        for feature, feature_args in features_set.items():
            for config_list, config_dict in zip(configs_list, configs) :
                args = th_args + feature_args + config_list + common_args
                identifier = f"{config_dict['model']}_{feature}_{th}-{th}_el{config_dict['e_layers']}_dm{config_dict['d_model']}_nh{config_dict['n_heads']}_dl{config_dict['d_layers']}_ff{config_dict['d_ff']}"
                str_args = f"{i}, {identifier}, {time_limits[config_dict['model']]}," + ",".join(args)
                str_args = str_args.replace(' ','')
                # if (identifier not in exp_count or exp_count[identifier] < 3) :
                if config_dict['model'] not in ['Transformer','LSTM','Nonstationary_Transformer'] and feature == 'MS':
                    print(str_args)
                    i+=1
                    f.write(str_args)
                    f.write("\n")
        

0,DLinear_MS_16-16_el2_dm512_nh8_dl1_ff2048,02:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=4,--features=MS,--enc_in=61,--dec_in=61,--c_out=61,--model=DLinear,--dropout=0.1,--learning_rate=0.001,--optimizer=adamw,--wd=0.05,--e_layers=2,--d_model=512,--n_heads=8,--d_layers=1,--d_ff=2048,--task_name=long_term_forecast,--is_training=1,--root_path=/tmpdir/arnaud/dataset/USC/,--checkpoints=/tmpdir/arnaud/checkpoints/,--results_path=/tmpdir/arnaud/results/,--data_path=na,--data=USC,--des=Exp,--batch_size=32,--train_epochs=48,--patience=16,--embed=fixed,--consider_only_offense,--inverse,--itr=3
1,PatchTST_MS_16-16_el3_dm128_nh16_dl2_ff128,06:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=4,--features=MS,--enc_in=61,--dec_in=61,--c_out=61,--model=PatchTST,--dropout=0.2,--learning_rate=0.001,--optimizer=adam,--wd=0.05,--e_layers=3,--d_model=128,--n_heads=16,--d_layers=2,--d_ff=128,--task_name=long_term_forecast,--is_training=1,--root_path=/tmpdir/arna

### All jobs

In [50]:
with open(SCRIPT_PATH / 'all_jobs_configs.txt', "w") as f:
    i = 0
    for th in time_horizons :
        th_args = [f"--model_id='USC_{th}_{th}'", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//4}"]
        for feature, feature_args in features_set.items():
            for config_list, config_dict in zip(configs_list, configs) :
                args = th_args + feature_args + config_list + common_args
                identifier = f"{config_dict['model']}_{feature}_{th}-{th}_el{config_dict['e_layers']}_dm{config_dict['d_model']}_nh{config_dict['n_heads']}_dl{config_dict['d_layers']}_ff{config_dict['d_ff']}"
                str_args = f"{i}, {identifier}, {time_limits[config_dict['model']]}," + ",".join(args)
                str_args = str_args.replace(' ','')
                f.write(str_args)
                f.write("\n")
                i+=1

### Hyperparams comp between runs

In [51]:
ldf = pd.read_csv(LOG_FILE, index_col=0)
ldf['run'] = [1 if ind < CALMIP_START_INDEX[2] else 2 for ind in ldf.index]
model= 'iTransformer'
for seq_len in [16,24,32,48,64,96]:
    hp_cols = ['run','model','features','enc_in','seq_len','pred_len','learning_rate','dropout','optimizer','wd','e_layers','d_model','n_heads','d_layers','d_ff']
    indices = ldf[hp_cols].drop_duplicates().query(f"model == '{model}'").query("features == 'MS'").query(f"seq_len == {seq_len}").index
    display(ldf.loc[indices,hp_cols+['rmse','fde']])
    

,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
168,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,256,4,1,256,1.443561,2.323885
169,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,256,4,1,256,1.450601,2.332160
170,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,256,8,1,256,1.442348,2.322347
171,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,512,4,1,512,1.448953,2.330210
172,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,512,4,1,512,1.455063,2.341624
173,1,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,512,8,1,512,1.450734,2.334224
929,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,256,4,1,256,0.642176,1.198726
932,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,3,256,4,1,256,0.641774,1.195672
935,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,4,256,8,1,256,0.682622,1.258449
939,2,iTransformer,MS,61,16,16,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
143,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,256,4,1,256,1.995806,3.211073
144,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,256,4,1,256,2.001076,3.219440
146,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,4,256,8,1,256,1.992844,3.202685
148,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,512,4,1,512,1.996784,3.207222
151,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,512,4,1,512,2.003004,3.221637
154,1,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,4,512,8,1,512,1.997576,3.212950
963,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,256,4,1,256,1.512109,2.615828
969,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
973,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
976,2,iTransformer,MS,61,24,24,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
7,1,iTransformer,MS,61,32,32,0.001,0.1,adamw,0.01,2,256,8,1,512,2.431306,3.858518
62,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,256,4,1,256,2.436089,3.866682
63,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,256,4,1,256,2.432088,3.861075
64,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,256,8,1,256,2.435086,3.862606
65,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,512,4,1,512,2.436152,3.864174
66,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,512,4,1,512,2.438879,3.864929
67,1,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,512,8,1,512,2.439884,3.872389
989,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,2,256,4,1,256,1.734917,3.077791
996,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
997,2,iTransformer,MS,61,32,32,0.001,0.1,adam,0.05,4,256,8,1,256,1.741516,3.077881


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
252,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,256,4,1,256,3.096723,4.794221
255,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,256,4,1,256,3.096600,4.793936
256,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,256,8,1,256,3.097650,4.797655
257,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
258,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,512,4,1,512,3.102762,4.805478
260,1,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,512,8,1,512,3.094412,4.788723
1025,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1033,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,3,256,4,1,256,2.654699,4.384897
1034,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,4,256,8,1,256,2.711016,4.450923
1038,2,iTransformer,MS,61,48,48,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
284,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,256,4,1,256,3.677281,5.712017
285,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,256,4,1,256,3.675936,5.708266
286,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,256,8,1,256,3.684022,5.720809
287,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
288,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN
290,1,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,512,8,1,512,3.679391,5.716301
1064,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1065,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
1072,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
1073,2,iTransformer,MS,61,64,64,0.001,0.1,adam,0.05,4,256,8,1,256,3.238571,5.058647


,run,model,features,enc_in,seq_len,pred_len,learning_rate,dropout,optimizer,wd,e_layers,d_model,n_heads,d_layers,d_ff,rmse,fde
366,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,256,4,1,256,4.937085,7.675739
367,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,256,4,1,256,4.935859,7.676727
368,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,256,8,1,256,4.935022,7.671932
369,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
370,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,512,4,1,512,NaN,NaN
371,1,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,512,8,1,512,NaN,NaN
1096,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,256,4,1,256,NaN,NaN
1105,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,3,256,4,1,256,NaN,NaN
1106,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,4,256,8,1,256,4.349303,6.664582
1110,2,iTransformer,MS,61,96,96,0.001,0.1,adam,0.05,2,512,4,1,512,NaN,NaN
